In [2]:
import os
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt

%matplotlib inline

# mediapip
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# read temperature data from csv file
def read_infrared_data(file):
  lines = open(file, "r", encoding='utf-8')
  data = []
  for line in lines:
    vs = line.strip().split(",")
    row = [float(v) for v in vs]
    data.append(row)
  return data

# find tempeature info based on the points
def find_temp(csv_matrix, image_x, image_y, image_width, image_height):
  m_width = len(csv_matrix[0])
  m_height = len(csv_matrix)
  rate1 = m_width * 1.0 / image_width
  rate2 = m_height * 1.0 / image_height
  c_x = int(image_x * rate1) - 1
  c_y = int(image_y * rate2) - 1
  if c_x < 0:
    c_x = 0
  if c_y < 0:
    c_y = 0
  if c_x > m_width - 1:
    c_x = m_width - 1
  if c_y > m_height - 1:
    c_y = m_height - 1
  # print(f"cx={c_x},cy={c_y}")
  return csv_matrix[c_y][c_x], c_x, c_y

# a function to get fact bounding and key face landmarks from the image
def GetFaceInfo(optical_path,csv_path):
    count=0
    list_node=[]
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:
        
            image = cv2.imread(optical_path)
            img_matrix = read_infrared_data(csv_path)
            # print("matrix: ", len(img_matrix[0]), len(img_matrix))
            # print("shape: ",image.shape)
            image_height, image_width, _ = image.shape
            # print("shape: ", image_width, image_height)
            # Convert the BGR image to RGB and process it with MediaPipe Face Detection.
            results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

            # Draw face detections of each face.
            if not results.detections:
                # print("Not found!")
                return None
            annotated_image = image.copy()
            for detection in results.detections:
                # print(detection)
                # ('Nose tip:')
                # print(mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.NOSE_TIP))
                nose_tip=mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.NOSE_TIP)
                left_eye=mp_face_detection.get_key_point(detection,mp_face_detection.FaceKeyPoint.LEFT_EYE)
                right_eye = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.RIGHT_EYE)
                left_ear_tragion = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.LEFT_EAR_TRAGION)
                right_ear_tragion = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.RIGHT_EAR_TRAGION)
                mouth_center = mp_face_detection.get_key_point(detection, mp_face_detection.FaceKeyPoint.MOUTH_CENTER)
                # bounding_box=detection.relative_bounding_box
                box=detection.location_data.relative_bounding_box
                score=detection.score
                # print("box: ",box)
                # print("noise tip: ",nose_tip.x, nose_tip.y)
                # print("score: ",score)
                def convert_m_xy(d):
                    x = d.x * image_width
                    y = d.y * image_height
                    temp, c_x, c_y = find_temp(img_matrix, x, y, image_width, image_height)
                    return (c_x,c_y,temp)
                model={
                    "id":f"face-{count}",
                    "score":score,
                    "box": (int(box.xmin * image_width),int(box.ymin*image_height),int(box.width*image_width),int(box.height*image_height)),
                    "nose_tip":convert_m_xy(nose_tip),
                    "left_eye": convert_m_xy(left_eye),
                    "right_eye": convert_m_xy(right_eye),
                    "left_ear_tragion": convert_m_xy(left_ear_tragion),
                    "right_ear_tragion": convert_m_xy(right_ear_tragion),
                    "mouth_center":convert_m_xy(mouth_center)
                }
                list_node.append(model)
                count+=1

                mp_drawing.draw_detection(annotated_image, detection)
            # cv2.imshow('annotated',annotated_image)
            # plt.show(annotated_image)
            # cv2.waitKey()
    return list_node



In [3]:

from PIL import Image

def crop_image_optical(image_path,save_path):
    #cropping images
    img = Image.open(image_path) ## 打开chess.png文件，并赋值给img
    region = img.crop((144,175,792+144,1091+175))## 0,0表示要裁剪的位置的左上角坐标，50,50表示右下角。
    region = region.resize((1080,1440))
    # region_resized=region.resize((64,64))
    # cropped_image=crop_image(optical_path)
    region.save(save_path)
    return region

def resize_image(image_path,new_size,target_path):
     img = Image.open(image_path) ## 打开chess.png文件，并赋值给img
     region = img.resize(new_size)
     region.save(target_path)
     
     
def merge_image(img_path1,image_path2):

    img = Image.open(img_path1)
    img2 = Image.open(image_path2)
    merge = Image.blend(img, img2, 0.3)
    return merge

def merge_image_blocks(im1,im2,alpha=0.1):

    merge = Image.blend(im1, im2, alpha)
    return merge


In [66]:
# Create a datasets
from tqdm import tqdm
import os
from quickcsv.file import *
import shutil

dataset_root_path='datasets/flir_data'

list_filenames_optical=[]
list_filenames_csv=[]
list_file_ids=[]
list_filenames_thermal=[]
list_filenames_raw=[]
for category in os.listdir(dataset_root_path):
    for file in os.listdir(os.path.join(dataset_root_path,category,'optical')):
        name, ext = os.path.splitext(file)
        optical_path=os.path.join(dataset_root_path,category,'optical',file)
        csv_path=os.path.join(dataset_root_path,category,'csv',name+".csv")
        thermal_path=os.path.join(dataset_root_path,category,'thermal',name+".jpg")
        raw_path=os.path.join(dataset_root_path,category,'raw',name+".jpg")
        # print(optical_path)
        if os.path.exists(optical_path) and os.path.exists(csv_path):
            list_filenames_optical.append(optical_path)
            list_filenames_csv.append(csv_path)
            list_filenames_thermal.append(thermal_path)
            list_filenames_raw.append(raw_path)
            list_file_ids.append(name)


print("Number of files: ",len(list_file_ids))

target_root_path='datasets-processed/FLIR_with_same_size'

total_count=0
list_result=[]
idx=0
for name in tqdm(list_file_ids):
    img=Image.open(list_filenames_raw[idx])
    width, height=img.size
    if width < height:
        # print(name,width, height)
        info=GetFaceInfo(list_filenames_optical[idx],list_filenames_csv[idx])
        if info!=None:
            bbox=info[0]['box']
            # print(name,bounding_box)
            # print()
            
            offset_x=144
            offset_y=175
            ratio_x=1080*1.0/792
            ratio_y=1440*1.0/1091

            # resize the x,y
            x,y,w,h=bbox
            x=(x-offset_x)*ratio_x
            y=(y-offset_y)*ratio_y
            w=w*ratio_x
            h=h*ratio_y
            
            total_count+=1
            list_result.append({
                "name":name,
                "x":int(x),
                "y":int(y),
                "w":int(w),
                "h":int(h),
                "label":'face'
            })
            target_raw_path=f'{target_root_path}/raw/{name}.jpg'
            target_csv_path=f'{target_root_path}/csv/{name}.csv'
            target_thermal_path=f'{target_root_path}/thermal/{name}.jpg'
            target_optical_path=f'{target_root_path}/optical/{name}.jpg'
            
            '''
            if os.path.exists(target_raw_path):
                raise Exception('Should not have the existing files!')
            else:
                shutil.copy(list_filenames_raw[idx],target_raw_path)
                shutil.copy(list_filenames_csv[idx],target_csv_path)
                shutil.copy(list_filenames_thermal[idx],target_thermal_path)
                shutil.copy(list_filenames_optical[idx],target_optical_path)
            '''
            shutil.copy(list_filenames_raw[idx],target_raw_path)
            shutil.copy(list_filenames_csv[idx],target_csv_path)
            # shutil.copy(list_filenames_thermal[idx],target_thermal_path)
            shutil.copy(list_filenames_optical[idx],target_optical_path)
            resize_image(list_filenames_thermal[idx],(1080,1440),target_thermal_path)
    idx+=1
                
print("Total no.: ",total_count)

write_csv(f'{target_root_path}/list_face_detection.csv',list_result)
                


Number of files:  467


100%|██████████| 467/467 [01:29<00:00,  5.23it/s]

Total no.:  105
Write CSV: ['name', 'x', 'y', 'w', 'h', 'label']  -> (datasets-processed/FLIR_with_same_size/list_face_detection.csv)


In [5]:
from tqdm import tqdm
from PIL import Image
def crop_image_square(image_path,square):
    #cropping images
    img = Image.open(image_path) ## 打开chess.png文件，并赋值给img
    region = img.crop((0,0,square,square))## 0,0表示要裁剪的位置的左上角坐标，50,50表示右下角。
    # region_resized=region.resize((64,64))
    return region

root_path='datasets-processed/FLIR_with_same_size/thermal'
target_root_path='datasets-processed/FLIR_with_same_size/thermal_sq'
for file in tqdm(os.listdir(root_path)):
    file1=os.path.join(root_path,file)
    file2=os.path.join(target_root_path,file)
    r=crop_image_square(file1,square=1080)
    r.save(file2)

100%|██████████| 105/105 [00:01<00:00, 93.89it/s]


In [7]:
# An example to get face info using optical image and temperature's csv file
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from PIL import Image
import matplotlib.patches as mpatches

from PIL import Image
def crop_image_optical(image_path,save_path=None):
    #cropping images
    img = Image.open(image_path) ## 打开chess.png文件，并赋值给img
    region = img.crop((144,175,792+144,1091+175))## 0,0表示要裁剪的位置的左上角坐标，50,50表示右下角。
    region = region.resize((1080,1440))
    # region_resized=region.resize((64,64))
    # cropped_image=crop_image(optical_path)
    if save_path!=None:
        region.save(save_path)
    return region

image_id='FLIR_20220324_011639'

target_root_path='datasets-processed/FLIR_with_same_size'

optical_path=fr'{target_root_path}'+f'/optical/{image_id}.jpg'
csv_path=fr'{target_root_path}'+f'/csv/{image_id}.csv'
thermal_path=fr'{target_root_path}'+f'/thermal/{image_id}.jpg'
raw_path=fr'{target_root_path}'+f'/raw/{image_id}.jpg'

display(Image(file=raw_path))

result=GetFaceInfo(optical_path=optical_path,csv_path=csv_path)
print(result)

offset_x=144
offset_y=175
ratio_x=1080*1.0/792
ratio_y=1440*1.0/1091

bbox=result[0]['box']

# resize the x,y
x,y,w,h=bbox
x=(x-offset_x)*ratio_x
y=(y-offset_y)*ratio_y
w=w*ratio_x
h=h*ratio_y

# show in cropped image
c_image=crop_image_optical(optical_path)
w1,h1=c_image.size

plt.imshow(c_image)
# left, bottom, width, height = bbox
rect=mpatches.Rectangle((x,y),w,h, 
                        fill=False,
                        color="purple",
                       linewidth=2)
plt.axis("off")
plt.tight_layout()
plt.gca().add_patch(rect)

plt.show()
plt.axis("off")
plt.tight_layout()
rect1=mpatches.Rectangle((x,y),w,h, 
                        fill=False,
                        color="red",
                       linewidth=2)
t_image=Image.open(thermal_path)
w2,h2=t_image.size
plt.imshow(t_image)
plt.gca().add_patch(rect1)

print(w1,h1,h1/w1)
print(w2,h2,h2/w2)
print(640/480)

plt.show()

merged_img=merge_image_blocks(c_image,t_image,alpha=0.8)
plt.axis("off")
merged_img=merged_img.resize((1080,1440))
plt.imshow(merged_img)
rect2=mpatches.Rectangle((x,y),w,h, 
                        fill=False,
                        color="green",
                       linewidth=2)

plt.gca().add_patch(rect2)

plt.show()


TypeError: 'module' object is not callable

In [75]:
# Convert to 64x64
def convert_dataset_to_dim(dim):
    root_path='datasets-processed/FLIR_with_same_size/thermal_sq'
    target_root_path=f'datasets-processed/FLIR_with_same_size/thermal_{dim}'
    if not os.path.exists(target_root_path):
        os.mkdir(target_root_path)
    for file in tqdm(os.listdir(root_path)):
        src_file=os.path.join(root_path,file)
        resize_image(src_file,(dim,dim),os.path.join(target_root_path,file))
    list_item=read_csv('datasets-processed/FLIR_with_same_size/list_face_detection.csv')
    ratio=dim*1.0/1080
    for idx,item in enumerate(list_item):
        x=int(int(item['x'])*ratio)
        y=int(int(item['y'])*ratio)
        w=int(int(item['w'])*ratio)
        h=int(int(item['h'])*ratio)
        list_item[idx]['x']=int(x)
        list_item[idx]['y']=int(y)
        list_item[idx]['w']=int(w)
        list_item[idx]['h']=int(h)
        
    write_csv(f'datasets-processed/FLIR_with_same_size/list_face_detection_{dim}.csv',list_item)

convert_dataset_to_dim(128)

100%|██████████| 105/105 [00:01<00:00, 87.75it/s]

Read CSV: ['name', 'x', 'y', 'w', 'h', 'label']  <- (datasets-processed/FLIR_with_same_size/list_face_detection.csv)
Write CSV: ['name', 'x', 'y', 'w', 'h', 'label']  -> (datasets-processed/FLIR_with_same_size/list_face_detection_128.csv)
